# Example usage

Run this notebook locally or via free cloud platforms:

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/wgms-org/mb-interpolation/improvements?labpath=demo.ipynb)

## Import functions

In [1]:
import numpy as np
import pandas as pd

from mass_balance_interpolations import (
  calc_mass_balance_amplitude,
  interpolate_daily_balances
)

## Create a test dataset

In [2]:
columns=['Year', 'WINTER_BALANCE', 'SUMMER_BALANCE', 'ANNUAL_BALANCE']
data = [
  [2000, np.nan, np.nan, -500],
  [2001, np.nan, np.nan, -250],
  [2002, np.nan, np.nan, 0],
  [2003, np.nan, np.nan, 100],
  [2004, np.nan, np.nan, -1500],
  [2005, 1000, -1500, -500],
  [2006, 1000, -1250, -250],
  [2007, 750, -750, 0],
  [2008, 1500, -1400, 100],
  [2009, 1000, -2500, -1500]
]
df = pd.DataFrame(data, columns=columns)
df

,Year,WINTER_BALANCE,SUMMER_BALANCE,ANNUAL_BALANCE
0,2000,NaN,NaN,-500
1,2001,NaN,NaN,-250
2,2002,NaN,NaN,0
3,2003,NaN,NaN,100
4,2004,NaN,NaN,-1500
5,2005,1000.0,-1500.0,-500
6,2006,1000.0,-1250.0,-250
7,2007,750.0,-750.0,0
8,2008,1500.0,-1400.0,100
9,2009,1000.0,-2500.0,-1500


## Calculate key statistics

Number of annual observations:

In [3]:
len(df)

10

Mean balances:

In [4]:
df.mean()

Year              2004.5
WINTER_BALANCE    1050.0
SUMMER_BALANCE   -1480.0
ANNUAL_BALANCE    -430.0
dtype: float64

Mass balance amplitude:

In [5]:
calc_mass_balance_amplitude(df)

1265.0

Total annual balance:

In [6]:
df['ANNUAL_BALANCE'].sum()

-4300

## Interpolate mass balance

Interpolate daily balances, based on seasonal balances if available, else based on annual balance and amplitude.

In [8]:
daily = interpolate_daily_balances(
  df,
  alpha=calc_mass_balance_amplitude(df),
  winter_fraction=8/12
)
daily

,BALANCE
DATE,
1999-10-01,-1.310241
1999-10-02,-1.198491
1999-10-03,-1.086770
1999-10-04,-0.975094
1999-10-05,-0.863484
...,...
2009-09-26,-3.741875
2009-09-27,-2.912938
2009-09-28,-2.082058


Calculate the total daily balance over the entire record. It should be equal to the total annual balance calculated above for the same period.

In [9]:
daily['BALANCE'].sum()

-4300.0

_So long, and thanks for all the fish_ (Douglas Adams)